## Open world : Decision Tree


In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS = 950

print("Loading data...")
with open("/content/sample_data/mon_standard.pkl", "rb") as file:
    data = pickle.load(file)

X_timestamps = []  # Packet timestamps
X_packet_sizes = []  # Packet sizes
X_cum_sizes = []  # Cumulative packet size
y_monitored = []  # Labels

for i in range(TOTAL_URLS):
    label = i if USE_SUBLABEL else i // URL_PER_SITE
    for sample in data[i]:
        timestamps = np.empty(len(sample), dtype=np.float32)
        packet_sizes = np.empty(len(sample), dtype=np.int16)
        cum_sizes = np.empty(len(sample), dtype=np.int32)

        cumulative_sum = 0
        for j, c in enumerate(sample):
            dr = 1 if c > 0 else -1
            timestamps[j] = abs(c)
            packet_sizes[j] = dr * 512
            cumulative_sum += packet_sizes[j]
            cum_sizes[j] = cumulative_sum

        X_timestamps.append(timestamps)
        X_packet_sizes.append(packet_sizes)
        X_cum_sizes.append(cum_sizes)
        y_monitored.append(label)

print(f"Total monitored samples: {len(y_monitored)}")

TOTAL_UNMON_URLS = 3000

print("Loading unmonitored datafile...")
with open("/content/sample_data/unmon_standard10_3000.pkl", "rb") as f:
    x = pickle.load(f)

for i in range(TOTAL_UNMON_URLS):
    sample = x[i]
    timestamps = np.empty(len(sample), dtype=np.float32)
    packet_sizes = np.empty(len(sample), dtype=np.int16)
    cum_sizes = np.empty(len(sample), dtype=np.int32)

    cumulative_sum = 0
    for j, c in enumerate(sample):
        dr = 1 if c > 0 else -1
        timestamps[j] = abs(c)
        packet_sizes[j] = dr * 512
        cumulative_sum += packet_sizes[j]
        cum_sizes[j] = cumulative_sum

    X_timestamps.append(timestamps)
    X_packet_sizes.append(packet_sizes)
    X_cum_sizes.append(cum_sizes)
    y_monitored.append(-1)  # Label unmonitored data as -1

print(f"Total combined samples: {len(y_monitored)}")

# padding
max_length = max(len(seq) for seq in X_timestamps)

def pad_sequences(sequences, maxlen):
    padded_sequences = np.zeros((len(sequences), maxlen), dtype=np.float32)
    for i, seq in enumerate(sequences):
        padded_sequences[i, :len(seq)] = seq[:maxlen]
    return padded_sequences

X_timestamps_padded = pad_sequences(X_timestamps, max_length)
X_packet_sizes_padded = pad_sequences(X_packet_sizes, max_length)
X_cum_sizes_padded = pad_sequences(X_cum_sizes, max_length)

# feature combination
X_combined = np.hstack((X_timestamps_padded, X_packet_sizes_padded, X_cum_sizes_padded))

# data split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_monitored, test_size=0.2, random_state=42)

# Binary Classification (Monitored vs Unmonitored)
binary_y_train = np.where(np.array(y_train) == -1, -1, 1)  # Monitored(1), Unmonitored(-1)
binary_y_test = np.where(np.array(y_test) == -1, -1, 1)

# Decision Tree Binary Classification
print("Training Open-World Binary Decision Tree model...")
dt_binary_model = DecisionTreeClassifier(random_state=42, class_weight="balanced")
dt_binary_model.fit(X_train, binary_y_train)

binary_y_pred_dt = dt_binary_model.predict(X_test)
binary_accuracy_dt = accuracy_score(binary_y_test, binary_y_pred_dt)

print(f"Binary Classification Accuracy (Decision Tree): {binary_accuracy_dt:.4f}")
print("\nBinary Classification Report (Decision Tree):")
print(classification_report(binary_y_test, binary_y_pred_dt))

# Multi-Class Classification (Monitored and Unmonitored)
print("Training Open-World Multi-Class Decision Tree model...")
dt_multi_model = DecisionTreeClassifier(random_state=42, class_weight="balanced")
dt_multi_model.fit(X_train, y_train)

multi_y_pred_dt = dt_multi_model.predict(X_test)
multi_accuracy_dt = accuracy_score(y_test, multi_y_pred_dt)

print(f"Multi-Class Classification Accuracy (Decision Tree): {multi_accuracy_dt:.4f}")
print("\nMulti-Class Classification Report (Decision Tree):")
print(classification_report(y_test, multi_y_pred_dt))


Loading data...
Total monitored samples: 19000
Loading unmonitored datafile...
Total combined samples: 22000
Training Open-World Binary Decision Tree model...
Binary Classification Accuracy (Decision Tree): 0.9998

Binary Classification Report (Decision Tree):
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       590
           1       1.00      1.00      1.00      3810

    accuracy                           1.00      4400
   macro avg       1.00      1.00      1.00      4400
weighted avg       1.00      1.00      1.00      4400

Training Open-World Multi-Class Decision Tree model...
Multi-Class Classification Accuracy (Decision Tree): 0.8473

Multi-Class Classification Report (Decision Tree):
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00       590
           0       0.84      0.70      0.76        30
           1       0.91      0.89      0.90        44
           2       0.83     

## Open world : Random Forest

In [1]:
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS = 950

print("Loading monitored datafile...")
with open("/content/sample_data/mon_standard.pkl", "rb") as file:
    data = pickle.load(file)

X_timestamps = []  # Packet timestamps
X_packet_sizes = []  # Packet sizes
X_cum_sizes = []  # Cumulative packet size
y_monitored = []  # Labels

for i in range(TOTAL_URLS):
    label = i if USE_SUBLABEL else i // URL_PER_SITE
    for sample in data[i]:
        timestamps = np.empty(len(sample), dtype=np.float32)
        packet_sizes = np.empty(len(sample), dtype=np.int16)
        cum_sizes = np.empty(len(sample), dtype=np.int32)

        cumulative_sum = 0
        for j, c in enumerate(sample):
            dr = 1 if c > 0 else -1
            timestamps[j] = abs(c)
            packet_sizes[j] = dr * 512
            cumulative_sum += packet_sizes[j]
            cum_sizes[j] = cumulative_sum

        X_timestamps.append(timestamps)
        X_packet_sizes.append(packet_sizes)
        X_cum_sizes.append(cum_sizes)
        y_monitored.append(label)

print(f"Total monitored samples: {len(y_monitored)}")

TOTAL_UNMON_URLS = 3000

print("Loading unmonitored datafile...")
with open("/content/sample_data/unmon_standard10_3000.pkl", "rb") as f:
    x = pickle.load(f)

for i in range(TOTAL_UNMON_URLS):
    sample = x[i]
    timestamps = np.empty(len(sample), dtype=np.float32)
    packet_sizes = np.empty(len(sample), dtype=np.int16)
    cum_sizes = np.empty(len(sample), dtype=np.int32)

    cumulative_sum = 0
    for j, c in enumerate(sample):
        dr = 1 if c > 0 else -1
        timestamps[j] = abs(c)
        packet_sizes[j] = dr * 512
        cumulative_sum += packet_sizes[j]
        cum_sizes[j] = cumulative_sum

    X_timestamps.append(timestamps)
    X_packet_sizes.append(packet_sizes)
    X_cum_sizes.append(cum_sizes)
    y_monitored.append(-1)  # Label unmonitored data as -1

print(f"Total combined samples: {len(y_monitored)}")

# padding
max_length = max(len(seq) for seq in X_timestamps)

def pad_sequences(sequences, maxlen):
    padded_sequences = np.zeros((len(sequences), maxlen), dtype=np.float32)
    for i, seq in enumerate(sequences):
        padded_sequences[i, :len(seq)] = seq[:maxlen]
    return padded_sequences

X_timestamps_padded = pad_sequences(X_timestamps, max_length)
X_packet_sizes_padded = pad_sequences(X_packet_sizes, max_length)
X_cum_sizes_padded = pad_sequences(X_cum_sizes, max_length)

# feature combination
X_combined = np.hstack((X_timestamps_padded, X_packet_sizes_padded, X_cum_sizes_padded))

# data split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_monitored, test_size=0.2, random_state=42)

# Binary Classification (Monitored vs Unmonitored)
binary_y_train = np.where(np.array(y_train) == -1, -1, 1)  # Monitored(1), Unmonitored(-1)
binary_y_test = np.where(np.array(y_test) == -1, -1, 1)

# Random Forest Binary Classification
print("Training Open-World Binary Random Forest model...")
rf_binary_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_binary_model.fit(X_train, binary_y_train)

binary_y_pred = rf_binary_model.predict(X_test)
binary_accuracy = accuracy_score(binary_y_test, binary_y_pred)

print(f"Binary Classification Accuracy (Random Forest): {binary_accuracy:.4f}")
print("\nBinary Classification Report (Random Forest):")
print(classification_report(binary_y_test, binary_y_pred))

# Multi-Class Classification (Monitored and Unmonitored)
print("Training Open-World Multi-Class Random Forest model...")
rf_multi_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_multi_model.fit(X_train, y_train)

multi_y_pred = rf_multi_model.predict(X_test)
multi_accuracy = accuracy_score(y_test, multi_y_pred)

print(f"Multi-Class Classification Accuracy (Random Forest): {multi_accuracy:.4f}")
print("\nMulti-Class Classification Report (Random Forest):")
print(classification_report(y_test, multi_y_pred))


Loading monitored datafile...
Total monitored samples: 19000
Loading unmonitored datafile...
Total combined samples: 22000
Training Open-World Binary Random Forest model...
Binary Classification Accuracy (Random Forest): 0.9932

Binary Classification Report (Random Forest):
              precision    recall  f1-score   support

          -1       1.00      0.95      0.97       590
           1       0.99      1.00      1.00      3810

    accuracy                           0.99      4400
   macro avg       1.00      0.98      0.99      4400
weighted avg       0.99      0.99      0.99      4400

Training Open-World Multi-Class Random Forest model...
Multi-Class Classification Accuracy (Random Forest): 0.8925

Multi-Class Classification Report (Random Forest):
              precision    recall  f1-score   support

          -1       0.74      0.95      0.83       590
           0       0.96      0.90      0.93        30
           1       0.89      0.75      0.81        44
           2  